In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import os
import pandas as pd
import json
import tensorflow_models as tfm

2023-08-17 16:33:48.135620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 16:33:48.790168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATASET_DIR = "../data/asl-fingerspelling"
TRAIN_LANDMARKS_PATH = DATASET_DIR+os.sep+"train_landmarks"
SUPPLEMENTAL_LANDMARKS_PATH = DATASET_DIR+os.sep+"supplemental_landmarks"
TRAIN_PATH = DATASET_DIR+os.sep+"train.csv"
TFRECORDS_PATH = DATASET_DIR+os.sep+"tfrecords"
list_files = np.array([TFRECORDS_PATH+os.sep+"train_landmarks"+os.sep+file_name for file_name in os.listdir(TFRECORDS_PATH+os.sep+"train_landmarks/")])

# save_tfrecords(train_df)
# I'm only interested in the hand coordinates, x and y
landmark_sequence = pd.read_parquet(DATASET_DIR+os.sep+"train_landmarks/5414471.parquet").reset_index()

In [3]:


def get_columns(df):
    columns_right_hand = [column for column in df.columns if "x_right_hand" in column or "y_right_hand" in column]
    columns_left_hand = [column for column in df.columns if "x_left_hand" in column or "y_left_hand" in column]
    return columns_right_hand + columns_left_hand

FEATURE_COLUMNS = get_columns(landmark_sequence)

In [4]:
import math

N = list_files.shape[0]*512

train_split = math.ceil(N*0.8)
validation_split = N - train_split

number_of_train_files = math.ceil(train_split/512)

np.random.shuffle(list_files)

train_files = list_files[:number_of_train_files]
val_files = list_files[number_of_train_files:]

print(f"train split: {train_files.shape[0]*512} | val split: {val_files.shape[0]*512}")

train split: 28160 | val split: 6656


In [5]:
val_files[:10]

array(['../data/asl-fingerspelling/tfrecords/train_landmarks/2072876091.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/450474571.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1497621680.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/532011803.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1969985709.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/296317215.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1726141437.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/175396851.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/614661748.tfrecord',
       '../data/asl-fingerspelling/tfrecords/train_landmarks/1358493307.tfrecord'],
      dtype='<U72')

In [6]:
with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    char_to_num=json.load(f)

In [7]:
VOCAB_SIZE = len(char_to_num.items())

In [8]:
char_to_num

{' ': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '=': 27,
 '?': 28,
 '@': 29,
 '[': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '~': 58}

In [9]:
len([w for w, _ in char_to_num.items()])

59

In [10]:
# Add pad_token, start pointer and end pointer to the dict
start_token = "<"
end_token = ">"
pad_token = "P"
space_token = " "
unk_token = "[UNK]"
pad_token_idx = 0
space_token_idx = 59
unk_token_idx = -1
start_token_idx = 60
end_token_idx = 61

char_to_num[pad_token] = pad_token_idx
char_to_num[space_token] = space_token_idx
char_to_num[unk_token] = unk_token_idx
char_to_num[start_token] = start_token_idx
char_to_num[end_token] = end_token_idx
num_to_char = {j:i for i,j in char_to_num.items()}

In [11]:
TARGET_MAX_LENGHT = 32

def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    
    return landmarks, phrase

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(char_to_num.keys()),
        values=list(char_to_num.values()),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
)

itext_vectorizer = tf.keras.layers.TextVectorization(
    output_sequence_length=TARGET_MAX_LENGHT,
    output_mode="int",
    standardize=None,
    split="character",
    vocabulary=[w for w, _ in char_to_num.items()]
)

def convert_fn(landmarks, phrase):
    # Add start and end pointers to phrase.
    phrase_with_start_end_token = start_token + phrase + end_token
    print(phrase_with_start_end_token)
    phrase_splited = tf.strings.bytes_split(phrase_with_start_end_token)
    print(phrase_splited)
    phrase_with_indexes = table.lookup(phrase_splited)
    print(phrase_with_indexes)
    # Vectorize and add padding.
    # 65 added the UNK token
    phrase_vectorized = tf.pad(
                                phrase_with_indexes,
                                # paddings=[[0, len([w for w, _ in num_to_char.items()])-tf.shape(phrase_with_indexes)[0]]],
                                paddings=[[0, 32]],
                                mode = 'CONSTANT',
                                constant_values = pad_token_idx
                        )
    return landmarks, phrase_vectorized

def vectorizer_preprocess(landmarks, phrase):
    return landmarks, itext_vectorizer(phrase)

2023-08-17 16:33:52.131859: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 16:33:52.150177: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 16:33:52.150333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
char_to_num[">"]

61

In [13]:
result = convert_fn([], "3 crecks")

<3 crecks>
tf.Tensor([b'<' b'3' b' ' b'c' b'r' b'e' b'c' b'k' b's' b'>'], shape=(10,), dtype=string)
tf.Tensor([60 18 59 34 49 36 34 42 50 61], shape=(10,), dtype=int32)


In [14]:
result

([],
 <tf.Tensor: shape=(42,), dtype=int32, numpy=
 array([60, 18, 59, 34, 49, 36, 34, 42, 50, 61,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)>)

In [15]:
train_dataset_raw_labels = tf.data.TFRecordDataset(train_files).map(decode_fn).padded_batch(64)
train_dataset = tf.data.TFRecordDataset(train_files).map(decode_fn).map(convert_fn).padded_batch(64)
train_dataset_textvectorizer = tf.data.TFRecordDataset(train_files).map(decode_fn).map(vectorizer_preprocess).padded_batch(64)

Tensor("add_1:0", shape=(), dtype=string)
Tensor("StringsByteSplit/RaggedGetItem/strided_slice_5:0", shape=(None,), dtype=string)
Tensor("None_Lookup/LookupTableFindV2:0", shape=(None,), dtype=int32)


In [16]:
next(iter(train_dataset_textvectorizer))[1][0].numpy()

array([45, 34, 14, 35, 34, 51, 51, 34, 16, 36, 34, 51, 48, 40, 48, 15, 49,
       41, 49,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [17]:
next(iter(train_dataset))[1][0].numpy()

array([60, 43, 32, 12, 33, 32, 49, 49, 32, 14, 34, 32, 49, 46, 38, 46, 13,
       47, 39, 47, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [18]:
"".join([num_to_char[index] for index in next(iter(train_dataset_textvectorizer))[1][0].numpy()])

'nc/dcttc1ectqiq0rjrPPPPPPPPPPPPP'

In [19]:
"".join([num_to_char[index] for index in next(iter(train_dataset))[1][0].numpy()])

'<la-barra/carogo.php>PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP'

In [20]:
next(iter(train_dataset_raw_labels))[1][0].numpy()

b'la-barra/carogo.php'

In [21]:
num_to_char

{59: ' ',
 1: '!',
 2: '#',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: '(',
 8: ')',
 9: '*',
 10: '+',
 11: ',',
 12: '-',
 13: '.',
 14: '/',
 15: '0',
 16: '1',
 17: '2',
 18: '3',
 19: '4',
 20: '5',
 21: '6',
 22: '7',
 23: '8',
 24: '9',
 25: ':',
 26: ';',
 27: '=',
 28: '?',
 29: '@',
 30: '[',
 31: '_',
 32: 'a',
 33: 'b',
 34: 'c',
 35: 'd',
 36: 'e',
 37: 'f',
 38: 'g',
 39: 'h',
 40: 'i',
 41: 'j',
 42: 'k',
 43: 'l',
 44: 'm',
 45: 'n',
 46: 'o',
 47: 'p',
 48: 'q',
 49: 'r',
 50: 's',
 51: 't',
 52: 'u',
 53: 'v',
 54: 'w',
 55: 'x',
 56: 'y',
 57: 'z',
 58: '~',
 0: 'P',
 -1: '[UNK]',
 60: '<',
 61: '>'}

In [22]:
next(iter(train_dataset_textvectorizer))

(<tf.Tensor: shape=(64, 294, 84), dtype=float32, numpy=
 array([[[0.28413114, 0.36007503, 0.44180965, ..., 0.        ,
          0.        , 0.        ],
         [0.26482275, 0.3005719 , 0.36837673, ..., 0.        ,
          0.        , 0.        ],
         [0.23754913, 0.31357628, 0.39528942, ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.40615228, 0.4608809 , 0.47783646, ..., 0.        ,
          0.        , 0.        ],
         [0.39446467, 0.44435763, 0.4467729 , ..., 0.        ,
          0.        , 0.        ],
         [0.38341478, 0.4231695 , 0.41584805, ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.       

In [23]:
tf.math.not_equal(next(iter(train_dataset_textvectorizer))[0], 0)

<tf.Tensor: shape=(64, 294, 84), dtype=bool, numpy=
array([[[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, F

In [24]:
from Transformers.Decoder import TransformerDecoder
from Transformers.Encoder import TransformerEncoder
from Transformers.PositionalEncoding import BasicPositionalEmbeddings
from Transformers.DataLoader import SampledTestatN

In [25]:
class TokenEmbedding(tf.keras.layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, embedings_dim=64, **kwargs):
        super().__init__(**kwargs)
        self.num_vocab = num_vocab
        self.maxlen = maxlen
        self.embedings_dim = embedings_dim
        
        self.emb = tf.keras.layers.Embedding(input_dim=num_vocab, output_dim=embedings_dim, mask_zero=True)
        self.pos_emb = tfm.nlp.layers.PositionEmbedding(maxlen, initializer='glorot_uniform', seq_axis=1)

    def call(self, x, **kwargs):      
        return self.pos_emb(self.emb(x))
        # return self.emb(x)

    def compute_mask(self, inputs, mask=None):
        # This will return a boolean matrix where is False when found a 0 in that position
        return tf.math.not_equal(inputs, 0)

    def get_build_config(self):
        return {"num_vocab": self.emb,
                "maxlen": self.maxlen,
                "embedings_dim": self.embedings_dim}


class LandmarkEmbedding(tf.keras.layers.Layer):
    def __init__(self, embedings_dim=64, **kwargs):
        super().__init__(**kwargs)
        self.embedings_dim = embedings_dim

        self.conv1 = tf.keras.layers.Conv1D(
            embedings_dim, 5, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            embedings_dim, 5, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1DTranspose(
            embedings_dim, 5, strides=2, padding="same", activation="relu"
        )
        self.conv4 = tf.keras.layers.Conv1DTranspose(
            embedings_dim, 5, strides=2, padding="same", activation="relu"
        )
        self.global_max_pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.positional_encoding = tfm.vision.layers.PositionalEncoding()

    def call(self, x, **kwargs):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        # x = self.global_max_pooling(x)
        
        return x #self.positional_encoding(x, output_states=False)
    
    # def compute_mask(self, inputs, mask=None):
    #     print("input: ", inputs.shape)
    #     print("mask: ", mask.shape if mask is not None else "no mask")
    #     # This will return a boolean matrix where is False when found a 0 in that position
    #     print("mask new: ", tf.math.not_equal(inputs, 0).shape)

    #     return tf.math.not_equal(inputs, 0)

    def get_build_config(self):
        return {"embedings_dim": self.embedings_dim}


In [26]:
# tfm.vision.layers.PositionalEncoding()(next(iter(train_dataset_textvectorizer))[0])

In [27]:
#  lme = LandmarkEmbedding(embedings_dim=128)
#  lme(next(iter(train_dataset_textvectorizer))[0])

In [28]:
# next(iter(train_dataset_textvectorizer))[1]

In [29]:
#  lme = TokenEmbedding(num_vocab=VOCAB_SIZE, embedings_dim=DIM_EMBEDDINGS, maxlen=TARGET_MAX_LENGHT)
#  lme(next(iter(train_dataset_textvectorizer))[1])

In [30]:
TARGET_MAX_LENGHT

32

In [31]:
import math

DIM_EMBEDDINGS = 256
ATTENTION_HEADS = 1
VOCAB_SIZE = len([w for w, _ in num_to_char.items()])
LATENT_DIMS = math.ceil(DIM_EMBEDDINGS/ATTENTION_HEADS)

source_input = tf.keras.Input(shape=(None, 84), dtype="float32", name="source")
target_input = tf.keras.Input(shape=(TARGET_MAX_LENGHT,), dtype="int64", name="target")

source_emb = LandmarkEmbedding(embedings_dim=DIM_EMBEDDINGS)(source_input)
target_emb = TokenEmbedding(embedings_dim=DIM_EMBEDDINGS, maxlen=TARGET_MAX_LENGHT+1)(target_input)

encoded_sequence = TransformerEncoder(num_heads=ATTENTION_HEADS, dim_emb=DIM_EMBEDDINGS, dim_dense=LATENT_DIMS)(source_emb)
decoded_sequence = TransformerDecoder(num_heads=ATTENTION_HEADS, dim_emb=DIM_EMBEDDINGS, dim_dense=LATENT_DIMS)(encoded_sequence, target_emb)
decoded_sequence = tf.keras.layers.Dropout(0.5)(decoded_sequence)

next_token = tf.keras.layers.Dense(units=VOCAB_SIZE, activation="softmax")(decoded_sequence)
model = tf.keras.Model(inputs=[source_input, target_input], outputs=next_token)

In [32]:
source_input = tf.random.uniform(shape=(8,))
source_emb = LandmarkEmbedding(embedings_dim=DIM_EMBEDDINGS)(source_input)
source_emb.shape

ValueError: Exception encountered when calling layer 'landmark_embedding_1' (type LandmarkEmbedding).

Input 0 of layer "conv1d_2" is incompatible with the layer: expected min_ndim=3, found ndim=1. Full shape received: (8,)

Call arguments received by layer 'landmark_embedding_1' (type LandmarkEmbedding):
  • x=tf.Tensor(shape=(8,), dtype=float32)
  • kwargs={'training': 'None'}

In [ ]:
target_input = tf.random.uniform(shape=(8, TARGET_MAX_LENGHT,))
target_emb = BasicPositionalEmbeddings(max_tokens=VOCAB_SIZE, dim_emb=DIM_EMBEDDINGS, max_seq_length=TARGET_MAX_LENGHT+1)(target_input)

target_emb.shape

In [ ]:
# num_heads=self.num_heads,
# key_dim=self.dim_emb,
# value_dim=self.dim_dense


mask = tf.math.not_equal(source_emb, 0)
mask = mask[:, tf.newaxis, :]

scores = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=LATENT_DIMS, value_dim=LATENT_DIMS)(source_emb, source_emb, source_emb, attention_mask=mask)
# scores = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=LATENT_DIMS, value_dim=DIM_EMBEDDINGS)(source_emb, source_emb, source_emb)

scores.shape

In [ ]:
encoded_sequence = TransformerEncoder(num_heads=8, dim_emb=DIM_EMBEDDINGS, dim_dense=LATENT_DIMS)(source_emb)
encoded_sequence.shape

In [53]:
tf.keras.backend.clear_session()

#input
# source_input = tf.keras.Input(shape=(None, 84), dtype="float32", name="source")
source = tf.keras.layers.Input(shape=(None, 84), dtype="float32", name="source")

# ENCODER START
# embeddings = LandmarkEmbedding(embedings_dim=DIM_EMBEDDINGS)(source_input)
encoded_source = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=LATENT_DIMS), merge_mode="sum")(source)
# encoded_source = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=LATENT_DIM), merge_mode="sum")(embeddings)
# A full sentence is reduce to the last state of a LSTM
# ENCODER END

# DECODER
# Encodded token and new generated token as inputs
# Reverse process like the conv1DTranspose with the segmentation model
target_input = tf.keras.Input(shape=(TARGET_MAX_LENGHT,), dtype="int64", name="target")

#learn a representation from the target (Vocabsize, latent_dim)
#Should this be MAX_TOKENS+1 for the END?
latent_space = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=DIM_EMBEDDINGS, mask_zero=True)(target)

#This will return (TARGET_SHAPE, LATENT_DIM)
# decoder = tf.keras.layers.LSTM(units=LATENT_DIM, return_sequences=True)
decoder = tf.keras.layers.GRU(units=LATENT_DIMS, return_sequences=True)

decoded_sentence = decoder(latent_space, initial_state=encoded_source)

decoded_sentence = tf.keras.layers.Dropout(0.5)(decoded_sentence)

target_next_step = tf.keras.layers.Dense(units=VOCAB_SIZE, activation="softmax")(decoded_sentence)
# DECODER

# CREATE MODEL ENCODER / DECODER
encoder_decoder_model = tf.keras.Model(inputs=[source, target], outputs=target_next_step)

# encoder_decoder_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5), metrics=[tf.keras.metrics.sparse_categorical_accuracy])
encoder_decoder_model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

: 

: 

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss")
save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath="../best_model/prototype", save_best_only=True, save_weights_only=True)

train_dataset = (tf.data.TFRecordDataset(train_files)
                    .shuffle(32)
                    .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)
                    .map(convert_fn, num_parallel_calls=tf.data.AUTOTUNE)
                    .map(lambda landmark, phrase: ({"source":landmark, "target":phrase[:-1]}, phrase[1:]))
                    .padded_batch(32)
                    .prefetch(tf.data.AUTOTUNE)
                )

val_dataset = (tf.data.TFRecordDataset(val_files)
                    .map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE)
                    .map(convert_fn, num_parallel_calls=tf.data.AUTOTUNE)
                    .map(lambda landmark, phrase: ({"source":landmark, "target":phrase[:-1]}, phrase[1:]))
                    .cache()
                    .padded_batch(32)
                    .prefetch(tf.data.AUTOTUNE)
                )

Tensor("add_1:0", shape=(), dtype=string)
Tensor("StringsByteSplit/RaggedGetItem/strided_slice_5:0", shape=(None,), dtype=string)
Tensor("None_Lookup/LookupTableFindV2:0", shape=(None,), dtype=int32)
Tensor("add_1:0", shape=(), dtype=string)
Tensor("StringsByteSplit/RaggedGetItem/strided_slice_5:0", shape=(None,), dtype=string)
Tensor("None_Lookup/LookupTableFindV2:0", shape=(None,), dtype=int32)


In [ ]:
encoder_decoder_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
encoder_decoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 target (InputLayer)         [(None, None)]               0         []                            
                                                                                                  
 source (InputLayer)         [(None, None, 84)]           0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            16128     ['target[0][0]']              
                                                                                                  
 bidirectional (Bidirection  (None, 256)                  525312    ['source[0][0]']              
 al)                                                                                          

In [ ]:
# model.fit(train_dataset, validation_data=val_dataset, epochs=1, callbacks=[early_stop, save_best_model])
encoder_decoder_model.fit(train_dataset, validation_data=val_dataset, epochs=1, callbacks=[early_stop])

2023-08-17 16:39:22.280443: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT8
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_41/output/_22'
2023-08-17 16:39:22.424257: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ca384c84d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-17 16:39:22.424274: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SU

InternalError: Graph execution error:

Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 3, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 256, 256, 1, 63, 32, 0] 
	 [[{{node gradients/cond_grad/gradients/cond/CudnnRNNV3_grad/CudnnRNNBackpropV3_tfg_inlined_gradients/cond_grad/If_0}}]]
	 [[PartitionedCall]] [Op:__inference_train_function_38768]

In [ ]:
model.load_weights("../best_model/prototype")

In [ ]:
model.save("../models/v1")

In [ ]:
model_loaded = tf.keras.models.load_model("../models/v1")